In [ ]:
Author: "Babak Badakhshan"
Created: 2024-12-24
Project: G-15 (Green 15-minute City)

#Let's install libraries
!pip install pandana osmnx

#Import libraries
import osmnx as ox
import pandana
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
ox.settings.log_console = False  # Disable OSMnx logging

# Configuration
cityname = 'Mashhad, Iran'
crs = 32640  # UTM Zone 40N for Mashhad

# Tags and OSM keys for each category
tags = {
"Education": ["college", "driving_school", "kindergarten", "language_school", "music_school", "school", "university"],
"Entertainment": ["arts_centre", "brothel", "casino", "cinema", "community_center", "conference_centre", "events_venue", "fountain",
         "gambling", "lovehotel","nightclub", "planetarium", "public_bookcase", "social_centre", "strip_club", "studio", "swinger_club",
         "theatre"],
"Grocery": ["alcohol", "bakery", "beverages", "brewing_supplies", "butcher", "cheese", "chocolate", "coffee", "confectionery", "convenience", "deli", "dairy",
            "farm", "frozen_food", "greengrocer", "health_food", "ice-cream", "pasta", "pastry", "seafood", "spices", "tea", "water",
            "supermarket", "department_store", "general", "kiosk", "mall"],
"Health": ["clinic", "dentist", "doctors", "hospital", "nursing_home", "pharmacy", "social_facility"],
"Posts_and_banks": ["atm", "bank", "bureau_de_change", "post_office"],
"Parks": ["park", "dog_park"],
"Sustenance": ["restaurant", "pub", "bar", "cafe", "fast-food", "food_court", "ice-cream", "biergarten"],
"Shops": ["department_store", "general", "kiosk", "mall", "wholesale", "baby_goods", "bag",
          "boutique", "clothes", "fabric", "fashion_accessories", "jewelry", "leather", "watches",
          "wool", "charity", "secondhand", "variety_store", "beauty", "chemist", "cosmetics", "erotic",
          "hairdresser", "hairdresser_supply", "hearing_aids", "herbalist", "massage", "medical_supply",
          "nutrition_supplements", "optician", "perfumery", "tattoo", "agrarian", "appliance", "bathroom_furnishing",
          "do-it-yourself", "electrical", "energy", "fireplace", "florist", "garden_centre", "garden_furniture",
          "gas", "glaziery", "groundskeeping", "hardware", "houseware", "locksmith", "paint", "security", "trade",
          "antiques", "bed", "candles", "carpet", "curtain", "doors", "flooring", "furniture", "household_linen",
          "interior_decoration", "kitchen", "lighting", "tiles", "window_blind", "computer", "electronics", "hifi",
          "mobile_phone", "radio-technics", "vacuum_cleaner", "bicycle", "boat", "car", "car_repair", "car_parts",
          "caravan", "fuel", "fishing", "golf", "hunting", "jet_ski", "military_surplus", "motorcycle", "outdoor",
          "scuba_diving", "ski", "snowmobile", "swimming_pool", "trailer", "tyres", "art", "collector", "craft",
          "frame", "games", "model", "music", "musical_instrument", "photo", "camera", "trophy", "video", "videogames",
          "anime", "books", "gift", "lottery", "newsagent", "stationery", "ticket", "bookmaker", "cannabis", "copy_node",
          "drycleaning", "e-cigarette", "funeral_directors", "laundry", "moneylender", "party", "pawnbroker", "pet",
          "pet_grooming", "pest_control", "pyrotechnics", "religion", "storage_rental", "tobacco", "toys", "travel_agency",
          "vacant", "weapons", "outpost"]

}

category_osm_keys = {
    'Education': 'amenity',
    'Entertainment': 'amenity',
    'Grocery': 'shop',
    'Health': 'amenity',
    'Posts_and_banks': 'amenity',
    'Parks': 'leisure',
    'Sustenance': 'amenity',
    'Shops': 'shop'
}

# Download street network
graph = ox.graph_from_place(cityname, network_type="walk")
graph = ox.projection.project_graph(graph, to_crs=crs)

# Configure walking parameters
walk_speed = 4.5  # km/h
for _, _, data in graph.edges(data=True):
    data['speed_kph'] = walk_speed
graph = ox.add_edge_travel_times(graph)

# Prepare network data
nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]

# Create Pandana network
network = pandana.Network(
    node_x=nodes['x'],
    node_y=nodes['y'],
    edge_from=edges['u'],
    edge_to=edges['v'],
    edge_weights=edges[['travel_time']]
)

# Retrieve POIs and calculate accessibility
maxdist = 200 * 60  # 200 minutes in seconds
nodes_gdf = gpd.GeoDataFrame(
    nodes,
    geometry=gpd.points_from_xy(nodes.x, nodes.y),
    crs=f"EPSG:{crs}"
)

for category in tags:
    osm_key = category_osm_keys[category]
    try:
        # Fetch POIs and calculate centroids
        gdf = ox.features_from_place(cityname, {osm_key: tags[category]}).to_crs(crs)
        if not gdf.empty:
            centroids = gdf.geometry.centroid
            # Set POIs in network
            network.set_pois(
                category=category,
                maxdist=maxdist,
                maxitems=1,
                x_col=centroids.x,
                y_col=centroids.y
            )
            # Calculate distances
            distances = network.nearest_pois(maxdist, category, num_pois=1)
            nodes_gdf[category] = distances[1]  # Nearest POI distance
        else:
            print(f"No POIs found for {category}")
    except Exception as e:
        print(f"Error processing {category}: {str(e)}")

# Plot accessibility for each category
for category in tags:
    if category in nodes_gdf.columns:
        fig, ax = plt.subplots(figsize=(10, 10))
        nodes_gdf.plot(column=category, ax=ax, legend=True,
                       legend_kwds={'label': f"Time to {category} (sec)"})
        ax.set_title(f"Walking Accessibility: {category}")
        plt.show()

# Save file as a gpkg (Optional)
nodes_gdf.to_file('Walking_Accessibility_Mashhad.gpkg')
